#  Ultimate Technologies Inc. Take-Home Challenge
## August Perez

Springboard Data Science track unit 37.2

In [ ]:
#imports

# data manipulation and math

import numpy as np
import scipy as sp
import scipy.stats as stats
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
import datetime as dt

# plotting and visualization

import matplotlib.pyplot as plt
import seaborn as sns

# modeling & pre-processing
    #commenting out since not necessary for this part of the project
#    import sklearn.model_selection
#    from sklearn.model_selection import train_test_split
#    from sklearn.model_selection import KFold
#    import sklearn.preprocessing
#    import sklearn.metrics

import warnings
    #reminder code so I can ignore warnings for single code cells:
    # warnings.filterwarnings('ignore')
    # warnings.resetwarnings()

# Part 1 - Exploratory data analysis
The attached logins.json file contains (simulated) timestamps of user logins in a particular geographic location. 

**Aggregate these login counts based on 15minute time intervals**, 

and **visualize and describe the resulting time series** of login counts in **ways that best characterize the underlying patterns of the demand**. 

Please **report/illustrate important features of the demand**, such as daily cycles. If there are **data quality issues, please report them**.

In [ ]:
#load data

logins_df = pd.read_json

# Part 2 ‐ Experiment and metrics design

# Part 3 ‐ Predictive modeling